# Wellpathpy tutorial

This document aims to provide a simple workthrough using `wellpathpy` showing:

- the abbreviations used in `wellpathpy`
- required imports
- loading a deviation
- loading a well header
- converting units
- choosing an md to tvd conversion method
- choosing a depth interval to resample to a given step
- exporting the results to \*.csv

## Abbreviations

- m: metres
- ft: feet
- md: measured depth
- inc: inclination
- azi: azimuth
- tvd: true vertical depth
- east_offset: horizontal distance away from wellhead towards the east
- north_offset: horizontal distance away from wellhead towards the north
- tvdss: true vertical depth subsea
- mE: horizontal distance in meters away from surface location towards the east
- mN: horizontal distance in meters away from surface location towards the north


## Table of contents

1. [Load a deviation](#Loading-a-deviation)
2. [Loading the well header](#Loading-the-well-header)
3. [Converting units](#Converting-units)
4. [Converting deviation surveys to positional logs](#Converting-deviation-surveys-to-positional-logs)
5. [Interpolate a deviation survey or positional log](#Interpolate-a-deviation-survey-or-positional-log)
6. [Exporting results](#Exporting-results)


## Imports

`wellpathpy` depends on [numpy](numpy.org), [pandas](pandas.pydata.org) and [pint](github.com/hgrecco/pint).

```
import numpy as np
import pandas as pd
import wellpathpy as wp
```

## Loading a deviation

There are two main ways of loading a deviation for use in `wellpathpy`, from a csv or within Python from a `pd.DataFrame` or a `np.ndarray`:

### 1. Loading a deviation from `*.csv`
A valid input file must be a `*.csv` file containing the columns: ('md', 'inc', 'azi') in that order, as shown in this example:  
```
md,inc,azi
0,0,244
10,11,220
50,43,254
150,78.5,254
252.5,90,359.9

```
- column headers are **required**
- `md` must increase monotonically
- as `inc` and `azi` cannot be distinguished numerically it is the user's responsibility to ensure the data are passed in in this order
- `inc` must be in range 0-180 (to allow for horizontal wells to climb)
- `azi` must be in range 0-360

You can then load them into `wellpathpy` using:

```
md, inc, azi = wp.read_csv(fname)
```

**Note**:  
The `wp.read_csv` function reads a deviation survey from a CSV file. The columns must be named `md`, `inc`, `azi`, and `md` must be strictly increasing. Some simple sanity checks are performed to reject bad CSVs. read_csv supports all options `pd.read_csv` supports. Only those columns named 'md', 'inc', 'azi' will be read.

### 2. Loading a deviation from a `pd.DataFrame` or a `np.ndarray`
In order to use `wellpathpy`, you will need three `np.ndarray` objects: `md`, `inc` and `azi`.
If you create these with pandas or NumPy you can still use `wellpathpy` as long as they fit the following criteria:
- `md`, `inc` and `azi` have the same `.shape` attribute
- `md` increases monotonically
- `inc` is in range 0-180
- `azi` is in range 0-360

## Loading the well header

Assuming the well header information in a \*.json file with this format and keys:
```
{
"datum": "kb",
"elevation_units": "m",
"elevation": 100.0,
"surface_coordinates_units": "m",
"surface_easting": 1000.0,
"surface_northing": 2000.0
}
```

we can load it with:

```
header = wp.read_header_json(fname)
```

## Converting units

By default, no unit conversions are run by `wellpathpy`, you therefore have the following options:

1. Import your deviations and headers in consistent SI units where:
    - `md`: meters
    - `inc`: degrees
    - `azi`: degrees
    - `elevation`: meters above mean sea level
    - `surface_easting`: meters east of reference point
    - `surface_northing`: meters north of reference point


2. Import deviations and headers in other units (e.g. feet) and convert to SI units:
    - `md`: feet
    - `inc`: degrees
    - `azi`: degrees
    - `elevation`: feet above mean sea level
    - `surface_easting`: feet east of reference point
    - `surface_northing`: feet north of reference point
    
**Important Notes:**  
- The units for [`elevation`, `surface_northing`, `surface_easting`] must be the same as the `md` units before any `md->tvd` calculations are run, otherwise you will get inconsistent results.
- `inc` and `azi` must always be passed as `degrees`, otherwise erroneous results will be returned.

### Conversion API

In order to convert `md`, `elevation`, `surface_easting` or `surface_northing` from 'ft' to 'm' where `elevation_units` and `surface_coordinates_units` are in 'ft' for example, run:

```
md               = wp.unit_convert(md, src='ft', dst='m')
elevation        = wp.unit_convert(header['elevation'], src=header['elevation_units'], dst='m')
surface_easting  = wp.unit_convert(header['surface_easting'],
                                   src=header['surface_coordinates_units'],
                                   dst='m')
surface_northing = wp.unit_convert(header['surface_northing'],
                                   src=header['surface_coordinates_units'], 
                                   dst='m')
```

We depend on [pint](github.com/hgrecco/pint) for the unit conversions. This means that you can add in your own units to the [unit registry](https://pint.readthedocs.io/en/latest/defining.html#programmatically) and then convert a quantity `data` from a unit [`ell`](https://en.wikipedia.org/wiki/Ell) for example to `meters` with the example below:

```
import pint
ureg             = pint.UnitRegistry()
ureg.define('ell = 0.6275 * meter = ell')
result           = wp.unit_convert(data, src='ell', dst='m', ureg=ureg)
```

## Converting deviation surveys to positional logs

`wellpathpy` provides the following methods to convert **deviation surveys** `md`, `inc`, `azi` into **positional logs** `tvd`, `northing`, `easting`:

### Recommended methods
These methods are most commonly used in drilling operations and are recommended for most cases:  
- `wp.mininum_curvature` **minimum curvature method**  
    This method uses angles from upper and lower end of survey interval to
    calculate a curve that passes through both survey points.  
    This curve is
    smoothed by use of the ratio factor defined by the tortuosity or dogleg
    of the wellpath.
    This method returns a dogleg severity calculated for a given course_length.
- `wp.radius_curvature` **radius of curvature method**  
    Calculate TVD using radius or curvature method.  
    **Caution**: this will yield unreliable results when data are closely spaced
    or when the borehole is straight but deviated.  
    This method uses angles from upper and lower end of survey interval to
    calculate a curve that passes through both survey points.

### Backup methods
These methods might be used for comparison to the recommended methods:  
- `wp.average_tan` **average tan method**  
    Calculate TVD using average tangential method.  
    This method averages the inclination and azimuth at the top and
    bottom of the survey interval before taking their sine and cosine,
    this average angle is used to estimate tvd.
- `wp.balanced_tan` **balanced tan method**
    Calculate TVD using balanced tangential method.  
    This method takes the sines and cosines of the inclination and azimuth
    at the top and bottom of the survey interval before averaging them,
    this average angle is used to estimate tvd.  
    This will provide a smoother curve than the ave_tan method but requires
    closely spaced survey stations to avoid errors.

### Not recommended methods
These methods are provided for completeness and in case a comparison must be made to an existing survey using these methods, but they are *not recommended*:  
- `wp.high_tan` **high tan method**  
    Calculate TVD using high tangential method.  
    This method takes the sines and cosines of the inclination and azimuth
    at the bottom of the survey interval to estimate tvd.  
    This method is **not recommended** as it can make gross tvd and offset
    errors in typical deviated wells.
- `wp.low_tan` **low tan method**  
    Calculate TVD using low tangential method.  
    This method takes the sines and cosines of the inclination and azimuth
    at the top of the survey interval to estimate tvd.  
    This method is **not recommended** as it can make gross tvd and offset
    errors in typical deviated wells.

### Usage
In order to use any of these functions, you can run the following code once you've imported your [deviation](#Loading-a-deviation) and [header](#Loading-the-well-header) and done any [unit conversion](#Converting-units) required as described above:

Recommended usage:
```
tvd, northing, easting, dls = wp.mininum_curvature(md, inc, azi, course_length=30)
tvd, northing, easting      = wp.radius_curvature(md, inc, azi)
```
Backup usage:
```
tvd, northing, easting = wp.average_tan(md, inc, azi)
tvd, northing, easting = wp.balanced_tan(md, inc, azi)
```
Not recommended:
```
tvd, northing, easting = wp.high_tan(md, inc, azi)
tvd, northing, easting = wp.low_tan(md, inc, azi)

```

## Interpolate a deviation survey or positional log
As deviation surveys are often not sampled at regular intervals, `wellpathpy` allows you to interpolate either a deviation survey or a positional log onto a given `md_step` or `tvd_step`. 

### Usage

- Interpolation of a deviation survey:
```
new_md, new_inc, new_azi           = interpolate_deviation(md, inc, azi, md_step=1)
```
- Interpolation of a positional log:
```
new_tvd, new_easting, new_northing = interpolate_position(tvd, easting, northing, tvd_step=1)
```

**Important Notes:**  
- `wp.interpolate_deviation()` should not be used before `md` ->` tvd` conversion. Rather, convert your deviation survey to a positional log first, and then interpolate them onto a new `tvd_step` if needed.  

- `wp.interpolate_position()` requires that the tvd values must be strictly increasing, i.e. this
method will **not work** on horizontal wells, use `wp.interpolate_deviation()` for those wells.  
In order to interpolate only a section of a positional log, you can slice into it and pass a monotonically increasing tvd section only.

- input arrays must not contain NaN values.  

## Exporting results
Once you have converted your deviation survey to a positional logs, you can write the results to a \*.csv file with:

```
wp.to_csv('path/to/file.csv', (tvd, northing, easting))
```